The purpose of this notebook is to open a particular HDF5 file and convert the data to an ndarray of floats.

In [2]:
import numpy as np
import pandas as pd
import os
import h5py

In [9]:
prs_path = '/fs/project/PAS1405/General/Kimmel_Chris/061120_8079m6A_IVTcarrierRNA_ligation_polyA.tombo.per_read_stats'
with h5py.File(prs_path, 'r') as hdf5file:
    block_stats = hdf5file['Statistic_Blocks']['Block_0']['block_stats'] 
    read_ids = hdf5file['Statistic_Blocks']['Block_0']['read_ids']
    '''There is another dataset in Block 0. It's called read_id_vals.
    When I wrote this code, read_id_vals was just a 1-dimensional dataset
    that assumed the value i at index i. As far as I can tell, read_id_vals
    is not important.
    '''
    
    print('block_stats variable:')
    display(block_stats)
    display(block_stats[0:4])
    
    bs_struct_array = np.asarray(block_stats)
    ri_struct_array = np.asarray(read_ids)
    '''I don't do anything with ri_struct_array in the Python notebook. I'm
    not even sure it's stored here as a structured array. And I don't know
    what information it stores in the HDF5 file. All we need to know about
    the read ids (I hope) is in the block_stats database.
    '''

bs_rec_array = np.rec.array(bs_struct_array)

print('bs_rec_array variable:')
display(bs_rec_array.dtype)
bs_rec_array[0:4]

block_stats variable:


<HDF5 dataset "block_stats": shape (28107436,), type "|V16">

array([(7075, 0.09857157, 34343), (7076, 0.37073309, 34343),
       (7077, 0.5976614 , 34343), (7078, 0.37987784, 34343)],
      dtype=[('pos', '<u4'), ('stat', '<f8'), ('read_id', '<u4')])

bs_rec_array variable:


dtype((numpy.record, [('pos', '<u4'), ('stat', '<f8'), ('read_id', '<u4')]))

rec.array([(7075, 0.09857157, 34343), (7076, 0.37073309, 34343),
           (7077, 0.5976614 , 34343), (7078, 0.37987784, 34343)],
          dtype=[('pos', '<u4'), ('stat', '<f8'), ('read_id', '<u4')])

In [10]:
pos_array = bs_rec_array['pos']
pos_set = set(pos_array)
print('number positions represented: {}'.format(len(pos_set)))
print('min position: {}'.format(min(pos_set)))
print('max position: {}'.format(max(pos_set)))
print('is the set of positions an interval? {}'.format(pos_set == {x for x in range(min(pos_set),max(pos_set)+1)}))
num_poss = len(pos_set)
min_pos = min(pos_set)

number positions represented: 1355
min position: 7060
max position: 8414
is the set of positions an interval? True


In [11]:
ri_array = bs_rec_array['read_id']
ri_set = set(ri_array)
print('number read ids: {}'.format(len(ri_set)))
print('min read: {}'.format(min(ri_set)))
print('max read: {}'.format(max(ri_set)))
print('is the set of read ids an interval? {}'.format(ri_set == {x for x in range(min(ri_set),max(ri_set)+1)}))
num_reads = len(ri_set)

number read ids: 35163
min read: 0
max read: 35162
is the set of read ids an interval? True


In [12]:
# Sort bs_rec_array. Maybe try sorting primarily by 'pos'. That might help us fill the table faster below.
# This step took me 20 seconds on a bs_rec_array with 28 million entries (Owens cluster, 1 node, 28 cores)
bs_rec_array_sorted = bs_rec_array.copy()
bs_rec_array_sorted.sort(order=['read_id','pos'], kind='mergegsort')

In [15]:
table = np.full((num_reads, num_poss), np.nan, np.dtype('f8'), order='C') # Interestingly, order='F' doesn't seem slower

# On a 28-core CPU in the Owens cluster this loop took a little over 3 minutes on a rec_array with 28 million entries:
for pos, stat, read in bs_rec_array_sorted:
    table[read, pos-min_pos] = stat
    
print('=============================================================')
print('=============================================================')
print('=============================================================')
print('=========================== DONE ============================')
print('=============================================================')
print('=============================================================')
print('=============================================================')

=========================== DONE ============================


In [33]:
sample = np.random.choice(bs_rec_array, 1000, replace=False)
if np.all([table[read, pos-min_pos] == stat for pos, stat, read in sample]):
    print('Table appears correct')
else:
    print('Table appears incorrect.')

Table appears correct


In [102]:
# This was part of a project to fill the table faster. I've tabled it.
# Each row of bs_rec_array_reads will correspond to a single read.

diffs = np.diff(bs_rec_array_sorted['read_id'])
assert np.count_nonzero(diffs) + 1 == num_reads

last_before_change = diffs.nonzero()[0]
# x is in last_before_change iff...
# ... the 'read_id' field of bs_rec_array_sorted changes between indices x and x+1

boundaries = np.concatenate(([0], last_before_change + 1, [len(bs_rec_array_sorted)]))
# now each read is a slice of bs_rec_array_sorted from boundaries[i] to boundaries[i+1]

# Make sure blocks between boundaries are constant
for st, sp in zip(boundaries, boundaries[1:]):
    assert len(np.nonzero(np.diff(read['read_id']))[0])==0

# Make sure each block contains a unique read: (be lazy and use the pigeonhole principle)
assert len(boundaries)-1 == num_reads

st = boundaries[:-1]
sp = boundaries[1:]

In [16]:
# Count_by_pos tells us the coverage at a particular genome position
count_by_pos = np.sum(~np.isnan(table), axis=0)

In [111]:
filename = 'per_read_csv_1.csv'

assert not os.path.exists(filename), 'Error: This program will not overwrite files. Delete {} and try again'.format(filename)
assert len(table) > 0, 'Trying to write a table with no entries'

row_labels = np.asarray(range(0, num_reads)).reshape(-1,1)
labeled_table = np.concatenate((row_labels, table), axis=1)

with open(filename, 'wb') as fh:
    
    header_entries = map(str, ['read_id'] + range(min_pos,min_pos + num_poss))
    header = ','.join(header_entries) + '\n'
    fh.write(header)
    
    for i, row in enumerate(table):
        row_entries = map(str, [i] + list(row))
        row = ','.join(row_entries)+'\n'
        fh.write(row)

# Checking CSV output

In [4]:
df = pd.read_csv(filename, index_col=0)

In [18]:
# Make sure the pandas dataframe equals the original data that went in.
# Note that we must conver the genome position from an int to a string to index into the dataframe

np.random.seed(855)
sample = np.random.choice(bs_rec_array, 10**4, replace=False)
for pos, stat, read in sample:
    if np.isclose(df.loc[read, str(pos)], stat):
        pass
    else:
        print('Table wrong at read {}, position {}'.format(read, pos))
        print('Should be {}, is {}.'.format(stat, df.loc[read, str(pos)]))
        break